### <font color="red">NOTE</font>

This was developed/tested in Google Colab, Python 3.10.12.

Also tested locally in VS Code, Python 3.9.16

# Project:  BCCR CRC/USD exchange rate (from different costarrican entities) extraction using webscraping

## Objective

The Central Bank of Costa Rica website where the latest information on CRC/USD echange rate for many official finantial entities is found at:

https://gee.bccr.fi.cr/IndicadoresEconomicos/Cuadros/frmConsultaTCVentanilla.aspx


The page also offers the option of checking the same information on previous dates

The objective of this project is to produce a file that contains the most up to date CRC/USD exchange rate for the diferent entities, that can later be used in other data science projects, as input data, for example as a new column, merging it to other tables/dataframes using the date index.



## Methodology

The steps that the code follows to fulfill the objective, are:


1. Download the entities exchange data from website, with a GET request
2. Extract the table using pandas.load_html
3. Get the headers and data from the table
4. Store the previous step items into a Pandas dataframe
5. Using BeautifulSoup, scrape the needed aspx parameters to be used in the next step
6. Send POST request (using aspx parameters from previous step as payload) to mimic the request that is done when clicking the calendar icon. Also, using BeautifulSoup, scrape the needed aspx parameters to be used in the next step
7. Send POST request (using aspx parameters from previous step as payload) to mimic the request that is done when clicking required date for yesterday 
8. Load the scraped data from previous step into a dataframe, using pandas.load_html
9. Repeat 6 thru 8 for N previous dates, going back one day at a time
10.
11.


---
## Description of used Packages and libraries

os - file operations such as renaming or getting the creation file\
requests - webscrapping of newer results\
shutil - High-level file operations, used to easily\ copy a file as a backup\
fastparquet - needed to produce a parquet file
matplotlib - basic plotting
pandas - dataframe operations \
numpy - required by Pandas\
datetime - timestamp operations\
python-magic / magic - needed to check/confirm file format
io - needed to wrap html string for pandas html read\
pytz - timezone operations for datetime
apt install -qq chromium-chromedriver  ----  installed at OS level as this is required by Selenium

---
## Findings/Lessons learned






### Initial page inspection

At the previous project, it was decided to download the historical data file of official BCCR exchange rate.    

The downloadable file that the bank provides here, only contains data for one specific date.

However, thru browser inspection is was found that each time the date selection is made and then applied, a POST request (with no query string) is sent back to the same URL with a distinctive parameter at the payload, and figured out that it represents the number of days that have passed since 2000-01-01:

For example, for 2024-08-05 the parameter seen is:

**EVENTARGUMENT: 8983**

It can be validated that the assumption made was in fact true, as follows:



```
from datetime import date
(date(2024, 8, 5) - date(2000, 1, 1)).days

8963
```

There is no need to download the XLS file for each day, we can just request the page for each day we need.   Therefore, Selenium is not needed here.

But, that parameter is not the only one seen at the payload, which is actually 13kb long.   For that reason, the payload, expressed as a non-url-encoded json, is placed inside a text file, except for that specific parameter above so it is easier to manage scraping multiple past days but just iterating on that value.


### Pandas reading from html object

The idea is NOT to download html files at all, but instead store the html content from a requests.text variable.

*   IO must be used as `pd.read_html(**io.StringIO(str**(page_request.text))` in order to avoid the following warning:

> FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.


*   read_html interprets a comma as a thousands separator, by default.  In this case we need to ignore that (using argument thousands=None) as the bank uses comma as decimal separator.



### Data Wrangling




###Data backup storage saving

Code was written to generate a backup of the generated file, as soon as it is downloaded.

By testing backups on different format files, in terms of storage space it was found that html file is the worst choice.   Parquet file is the one we can choose to make backups for the least amount of space.



---


## Enhancement opportunities


1.   File generation and download

By inspecting the extracted html, it was found that the file can actually be generated directly by modifying the query string values of the following URL

https://gee.bccr.fi.cr/indicadoreseconomicos/Cuadros/frmVerCatCuadro.aspx?CodCuadro=400&Idioma=1&FecInicial=2023/01/01&FecFinal=2024/07/19&Filtro=0&Exportar=True&Excel=True

Basically the initial and end dates can be modified at will to get the necessary file.

This can be done to avoid having to use logic (with Selenium) to simulate the clicking of the button to generate the file, so the code is smaller and quicker.
\
2.   Add code to download the full historical file, once a month.
\
3.   Add code to check if the updated exchange file is missing more than just one record (today), if it is, then complete the missing records from the fresh scraped data that should be done every day
\
4.   Clean the backup logic, the files at the end look a bit disorganized.  Some need timestamp at the filename






---

# Project Code



## 1.Preliminary tasks

### Packages,Libraries,Constants

Packages installation takes about 2 minutes

In [2]:
!pip -V

pip 24.2 from /home/milos/Documents/Proyectos/CienciaDeDatos/BCCR-tcdolar-entidades/venv/lib/python3.9/site-packages/pip (python 3.9)


In [3]:
!pip install --upgrade pip

In [536]:
!pip install --quiet matplotlib python-magic fastparquet requests pandas lxml bs4
# !apt-get update
# !apt install chromium-chromedriver
import os ,requests, shutil, time, magic, io, lxml, json, pytz, bs4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta
from io import StringIO
from bs4 import BeautifulSoup as bs

In [609]:
# Making sure locale is en_US so datetime/date outputs names of days/months in english
locale.setlocale(locale.LC_TIME, ('en_US', 'UTF-8'))

'en_US.UTF-8'

In [474]:
# Constants
DATASETS_PATH = './datasets'
DATASETS_TEMP_PATH = f'{DATASETS_PATH}/temp'
DATASETS_BACKUP_PATH = f'{DATASETS_PATH}/backups'
CURRENT_DATASET_BASEFILENAME = 'bccr_dol_exch_entities'
# JSON_FILE_NAME = 'payload_json_unencoded.txt'  #ended up not being needed
base_payload_json = {"__EVENTTARGET": "Calendar1", "__EVENTARGUMENT": "" , "__VIEWSTATE": "" ,\
  "__VIEWSTATEGENERATOR": "5CF5411C" ,  "__EVENTVALIDATION": "" ,\
    "imgCalendario.x": "7", "imgCalendario.y": "8" ,  "CtrlBuscar2:txtPalabras": ""}


### Checking existance of current dataset file

If if does not exist, then generate the directory where it is to be  placed at later with data from the past 365 days






In [6]:
# prompt: need an if statement, that if CURRENT_DATASET_BASEFILENAME exists, print OK, if not print NOK and also create the DATASETS_PATH path

if os.path.exists(f'{DATASETS_PATH}/{CURRENT_DATASET_BASEFILENAME}.parquet'):
  print ("OK")
else:
  print ("NOK")
  os.makedirs(DATASETS_PATH, exist_ok=True)


OK


In [7]:
if os.path.exists(f'{DATASETS_PATH}/{CURRENT_DATASET_BASEFILENAME}.parquet'):
  print(f"OK, {DATASETS_PATH}/{CURRENT_DATASET_BASEFILENAME}.parquet already exists")
else:
  print(f"There is no historical data file {DATASETS_PATH}/{CURRENT_DATASET_BASEFILENAME}.parquet")
  os.makedirs(DATASETS_PATH, exist_ok=True)


OK, ./datasets/bccr_dol_exch_entities.parquet already exists


## 2.Scraping today's entities dollar exchange rate

In [589]:
# Scraping html of the most recent data from the bank webpage

# Configuration

host='gee.bccr.fi.cr'
urlpath='IndicadoresEconomicos/Cuadros/frmConsultaTCVentanilla.aspx'
# Sending some headers to try and hide the scraper default values
hdrs={'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
      'Host' : f'{host}',
      'Accept-Language': 'en-US,en;q=0.9,es;q=0.8,es-CR;q=0.7,de;q=0.6',
      'Accept-Encoding': 'gzip, deflate, br, zstd'
      }

url_page = f'https://{host}/{urlpath}'

In [590]:
# Performing the actual request.  Today's data is done with a GET request
page_request = requests.get(url_page, headers=hdrs)

In [591]:
# some interesting options that could be used
page_request.status_code, page_request.reason ,  page_request.ok , page_request.url , \
page_request.headers['Content-Length'], page_request.headers['Date'] , page_request.encoding, \
 page_request.headers['Content-Type']

(200,
 'OK',
 True,
 'https://gee.bccr.fi.cr/IndicadoresEconomicos/Cuadros/frmConsultaTCVentanilla.aspx',
 '13266',
 'Fri, 16 Aug 2024 10:15:06 GMT',
 'utf-8',
 'text/html; charset=utf-8')

In [592]:
page_request.text[:300]

'\r\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">\r\n<HTML>\r\n\t<HEAD>\r\n\t\t<title>Tipo de cambio anunciado en ventanilla</title>\r\n\t\t<script type="text/javascript">var _gaq = _gaq || [];_gaq.push([\'_setAccount\', \'UA-25040215-3\']);_gaq.push([\'_trackPageview\']);(function() {var ga = document.'

### Building the dataframe from scraped html

In [593]:
# test with basic cleaning
pd.read_html(io.StringIO(str(page_request.text)) , thousands=None,
                            decimal=',' , header=0 )[2].head(3)

,Tipo de Entidad,Entidad Autorizada,Compra,Venta,Diferencial Cambiario,Última Actualización
0,Bancos públicos,Banco de Costa Rica,512.0,526.0,14.0,15/08/2024 12:03 a.m.
1,NaN,Banco Nacional de Costa Rica,508.0,522.0,14.0,14/08/2024 12:39 p.m.
2,NaN,Banco Popular y de Desarrollo Comunal,511.0,525.0,14.0,14/08/2024 03:55 p.m.


In [594]:
[s for s in pytz.all_timezones if 'Costa' in s]

['America/Costa_Rica']

In [595]:
datetime.now(pytz.timezone('America/Costa_Rica')).strftime('%Y-%m-%d')

'2024-08-16'

In [596]:
# -setting the header as the resulting line with index 0
# -recognizing the original decimal char as the comma, so dataframe is shown as usual with it as a dot
# -dropping rows made of NaN at every column
# -name the columns properly
# -extend the ent_type for those with NaN
# -apply datetime format to previous_updt column
# -insert 'date' column with current date (in Costa Rica as the bank is in that country)
#  as for all rows.

if page_request.ok:
  print('Request OK')
  cols=['date', 'dollar_buy','dollar_sale']
  cols=['ent_type', 'ent_name', 'dollar_buy','dollar_sale','b_s_diff','previous_updt']
  df_dol_ent = pd.read_html(io.StringIO(str(page_request.text)) , thousands=None,
                            decimal=',' , header=0 )[2]

  df_dol_ent.dropna(axis = 0, how = 'all', inplace = True)
  df_dol_ent.columns = cols
  df_dol_ent.ffill( inplace=True)
  df_dol_ent['previous_updt'] = pd.to_datetime(df_dol_ent['previous_updt'] , format='mixed',dayfirst=True)
  # df_dol_ent.insert(loc=0, column = 'dateUTC', value =  datetime.today().strftime('%Y-%m-%d'))
  today_CostaRica = datetime.now(pytz.timezone('America/Costa_Rica')).strftime('%Y-%m-%d')
  df_dol_ent.insert(loc=0, column = 'date', value = today_CostaRica)
  df_dol_ent['date']=pd.to_datetime(df_dol_ent['date'])

else:
  print('Request was NOT OK, received status code', page_request.status_code)

df_dol_ent.tail(7)
# df_dol_ent.head(7)


Request OK


,date,ent_type,ent_name,dollar_buy,dollar_sale,b_s_diff,previous_updt
31,2024-08-16,Casas de Cambio,Casa de Cambio Global Exchange,433.96,606.78,172.82,2024-08-14 21:12:00
32,2024-08-16,Casas de Cambio,Casa de Cambio Teledolar S. A.,508.00,531.00,23.00,2024-08-16 00:21:00
33,2024-08-16,Puestos de Bolsa,"BCT Valores, Puesto De Bolsa, S.A.",509.00,527.00,18.00,2024-08-14 12:22:00
34,2024-08-16,Puestos de Bolsa,"BN Valores S.A., Puesto de Bolsa",512.00,526.00,14.00,2024-08-14 08:12:00
35,2024-08-16,Puestos de Bolsa,Mercado Valores de Costa Rica Puesto de Bolsa,506.00,524.00,18.00,2024-08-14 12:45:00
36,2024-08-16,Puestos de Bolsa,PB Inversiones SAMA,507.00,523.00,16.00,2024-08-14 13:50:00
37,2024-08-16,Puestos de Bolsa,"Popular Valores, Puesto de Bolsa",513.00,527.00,14.00,2024-08-14 10:54:00


In [597]:
df_dol_ent.ent_type.unique() , df_dol_ent.ent_type.nunique() , df_dol_ent.ent_type.value_counts()

(array(['Bancos públicos', 'Bancos privados', 'Financieras',
        'Mutuales de Vivienda', 'Cooperativas', 'Casas de Cambio',
        'Puestos de Bolsa'], dtype=object),
 7,
 ent_type
 Bancos privados         11
 Cooperativas             9
 Puestos de Bolsa         5
 Financieras              4
 Casas de Cambio          4
 Bancos públicos          3
 Mutuales de Vivienda     2
 Name: count, dtype: int64)

In [598]:
df_dol_ent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 0 to 37
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           38 non-null     datetime64[ns]
 1   ent_type       38 non-null     object        
 2   ent_name       38 non-null     object        
 3   dollar_buy     38 non-null     float64       
 4   dollar_sale    38 non-null     float64       
 5   b_s_diff       38 non-null     float64       
 6   previous_updt  38 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(2)
memory usage: 2.4+ KB


#### Function

In [599]:
def dol_exch_ent_df_build (pagereq):
  # -setting the header as the resulting line with index 0
  # -know that there is no thousands separator so pandas does not get confused with the original commas
  # -recognizing the original decimal char as the comma, so dataframe is shown as usual with it as a dot
  # -dropping rows made of NaN at every column
  # -name the columns properly
  # -extend the ent_type to those with NaN
  # -apply datetime format to previous_updt column
  # -insert 'date' column with current date (in Costa Rica as the bank is in that country)
  
  if pagereq.ok:
    # print('Request OK')
    # cols=['date', 'dollar_buy','dollar_sale']
    cols=['ent_type', 'ent_name', 'dollar_buy','dollar_sale','b_s_diff','previous_updt']
    df_temp = pd.read_html(io.StringIO(str(pagereq.text)) , thousands=None,
                              decimal=',' , header=0 )[2]

    df_temp.dropna(axis = 0, how = 'all', inplace = True)
    df_temp.columns = cols
    df_temp.ffill( inplace=True)
    df_temp['previous_updt'] = pd.to_datetime(df_temp['previous_updt'] , format='mixed',dayfirst=True)
    # df_temp.insert(loc=0, column = 'dateUTC', value =  datetime.today().strftime('%Y-%m-%d'))
    today_CostaRica = datetime.now(pytz.timezone('America/Costa_Rica')).strftime('%Y-%m-%d')
    df_temp.insert(loc=0, column = 'date', value = today_CostaRica)
    df_temp['date']=pd.to_datetime(df_temp['date'])
    return df_temp

  else:
    print('Request was NOT OK, received status code', pagereq.status_code)

In [600]:
# function validation
df_dol_ent_today = dol_exch_ent_df_build(page_request)
df_dol_ent_today.head(3)

,date,ent_type,ent_name,dollar_buy,dollar_sale,b_s_diff,previous_updt
0,2024-08-16,Bancos públicos,Banco de Costa Rica,512.0,526.0,14.0,2024-08-15 00:03:00
1,2024-08-16,Bancos públicos,Banco Nacional de Costa Rica,508.0,522.0,14.0,2024-08-14 12:39:00
2,2024-08-16,Bancos públicos,Banco Popular y de Desarrollo Comunal,511.0,525.0,14.0,2024-08-14 15:55:00


In [601]:
df_dol_ent_today.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 0 to 37
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           38 non-null     datetime64[ns]
 1   ent_type       38 non-null     object        
 2   ent_name       38 non-null     object        
 3   dollar_buy     38 non-null     float64       
 4   dollar_sale    38 non-null     float64       
 5   b_s_diff       38 non-null     float64       
 6   previous_updt  38 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(2)
memory usage: 2.4+ KB


### Extracting the __EVENTVALIDATION and __VIEWSTATE aspx parameters from base page

Those values will be needed at the next stage, as extracting the exchange info from previous dates requires a POST with the aspx parameters values from the base page

In [602]:
# getting the soup
if page_request.ok:
  base_page_soup = bs(page_request.text)
else:
  print('Not Found. Status code was: ', page_request.status_code)

# extracting values of parameters
base_EVENTVALIDATION = base_page_soup.find('input', attrs={'name': '__EVENTVALIDATION'})['value']
base_VIEWSTATE = base_page_soup.find('input', attrs={'name': '__VIEWSTATE'})['value']

In [603]:
base_EVENTVALIDATION[:10] , base_EVENTVALIDATION[-10:] , base_VIEWSTATE[:10], base_VIEWSTATE[-10:]

('/wEdAAjH6P', 'dbLmTaco0x', '/wEPDwUJMj', 'v8xCvcRg8=')

#### Function

In [604]:
# function to extract those parameters, as doing this is going to be done repeatedly later

def get_eventvalidation_viewstate ( pagerequested ):
  if pagerequested.ok:
    page_soup = bs(pagerequested.text)
    # extracting values of parameters
    param_EVENTVALIDATION = page_soup.find('input', attrs={'name': '__EVENTVALIDATION'})['value']
    param_VIEWSTATE = page_soup.find('input', attrs={'name': '__VIEWSTATE'})['value']
    return param_EVENTVALIDATION , param_VIEWSTATE
  else:
    print('Not Found. Status code was: ', pagerequested.ok, page_request.status_code)  
    param_EVENTVALIDATION = 'unable to load page'
    param_VIEWSTATE = 'unable to load page'
    return param_EVENTVALIDATION , param_VIEWSTATE

In [605]:
# testing the function
base_EVENTVALIDATION , base_VIEWSTATE   = get_eventvalidation_viewstate (page_request)
# checking last 10 chars for each, as those are too big
base_EVENTVALIDATION[-10:] , base_VIEWSTATE[-10:] 

('dbLmTaco0x', 'v8xCvcRg8=')

## 3.Scraping previous date entities dollar exchange rate

This section's purpose it to understand and confirm how the POST request to get previous days data is to be made, successfully

In [371]:
# !ls pay*  
# commented as the payload txt file is no longer needed as we will extract the aspx params from base page

In [372]:
# this one did not work as data is already encoded and also NOT in json format
# with open('payload_json.txt','r') as f:
  # payld =  f'{f.read()}'
# payld
# type(payld)


In [373]:
# # get text dict from file, and then convert to a true dict object
# with open('payload_json_unencoded.txt','r') as f:
#   payld =  f.read()

# import json
# payld = json.loads(payld)
# payld
# # type(payld)

In [374]:
## NOT USED anymore as aspx params are being loaded from scraped page, instead hardcoded from a file

# ## Config is mostly the same as previous section. Adding the payload needed for the
# ## POST request that is done instead the GET request done before

# # url_page = f'https://{host}/{urlpath}'

# # When inspecting the browser behaviour, this request header is sent
# # Content-Type: application/x-www-form-urlencoded

# hdrs={'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
#       'Host' : f'{host}',
#       'Accept-Language': 'en-US,en;q=0.9,es;q=0.8,es-CR;q=0.7,de;q=0.6',
#       'Accept-Encoding': 'gzip, deflate, br, zstd',
#       # 'Content-Type': 'application/x-www-form-urlencoded'  #removed, requests library takes care of it
#       }

# # payload = { '__EVENTTARGET' : 'Calendar1' , '__EVENTARGUMENT' : '8983' }

# # with open('payload_json_unencoded.txt','r') as f:
# #   payld =  f.read()
# # import json
# # payld = json.loads(payld)

# # taken from recursively scraping section
# if os.path.exists(JSON_FILE_NAME):
#   print(f'Payload {JSON_FILE_NAME} exists locally, loading it into a variable...')
#   with open(JSON_FILE_NAME,'r') as f:
#     payld =  f.read()
#   payld = json.loads(payld)
# else:
#   print(f'Payload {JSON_FILE_NAME} does not exist locally, loading it into a variable from github rawfile...')
#   jsonfilereq = requests.get('https://raw.githubusercontent.com/lemilosm/bccr_dol_exc_entities_rate_history_webscraping/main/payload_json_unencoded.txt')
#   if jsonfilereq.ok:
#     payld = json.loads(jsonfilereq.text)
#     #storing the downloaded json data into a file for future runs to have it
#     with open(JSON_FILE_NAME,'w') as f:
#       f.write(str(payld).replace("'", '"')) #future loads from file require " instead '
#     print(f'{JSON_FILE_NAME} saved locally')
#   else:
#     print('Payload json file could not be read from github either.')

# # argument that controls the date, as described at the project's intro
# past_date_to_scrape = date(2024, 8, 5)
# past_date_to_scrape_inDays = (past_date_to_scrape - date(2000, 1, 1)).days

# payld['__EVENTARGUMENT'] = past_date_to_scrape_inDays
# # "__EVENTARGUMENT": "8983"

# past_date_to_scrape_inDays , payld['__EVENTTARGET'] ,payld['__VIEWSTATEGENERATOR'] , payld['__EVENTARGUMENT']

### Getting calendar page

In [375]:
# Base Json string to be used as payload for POST requests
# all parameters were observed to be constant, except for _EVENTVALIDATION and VIEWSTATE

#this one defined as a constant at the beginning of the code
# base_payload_json = {"__EVENTTARGET": "Calendar1", "__EVENTARGUMENT": "" , "__VIEWSTATE": "" ,\
#   "__VIEWSTATEGENERATOR": "5CF5411C" ,  "__EVENTVALIDATION": "" ,\
#     "imgCalendario.x": "7", "imgCalendario.y": "8" ,  "CtrlBuscar2:txtPalabras": " "}

In [376]:
payload_json = base_payload_json.copy()
# argument that controls the date, as described at the project's intro
past_date_to_scrape = date(2024, 8, 1)
past_date_to_scrape_inDays = (past_date_to_scrape - date(2000, 1, 1)).days
# __EVENTVALIDATION and __VIEWSTATE populated with info from previous section, and __EVENTARGUMENT from
#info of past date to be scraped
payload_json['__EVENTVALIDATION'] = base_EVENTVALIDATION
payload_json['__VIEWSTATE'] = base_VIEWSTATE
payload_json['__EVENTARGUMENT'] = past_date_to_scrape_inDays

In [377]:
for k in  ['__EVENTVALIDATION','__VIEWSTATE'] :
  val = payload_json[k][-5:]
  print(f'{k} : {val}')

__EVENTVALIDATION : aco0x
__VIEWSTATE : cRg8=


In [378]:
# payload_json

In [379]:
# Performing the actual request.  Page with calendar is obtained with a POST request

hdrs_post={'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
      'Host' : f'{host}',
      'Accept-Language': 'en-US,en;q=0.9,es;q=0.8,es-CR;q=0.7,de;q=0.6',
      'Accept-Encoding': 'gzip, deflate, br, zstd',
      # 'Content-Type': 'application/x-www-form-urlencoded'  #removed, requests library takes care of it
      }

page_request_calendar = requests.post(url_page, headers=hdrs_post , data=payload_json )

page_request_calendar.status_code, page_request_calendar.reason

(200, 'OK')

In [380]:
# req headers
page_request_calendar.request.headers

{'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive', 'Host': 'gee.bccr.fi.cr', 'Accept-Language': 'en-US,en;q=0.9,es;q=0.8,es-CR;q=0.7,de;q=0.6', 'Content-Length': '12484', 'Content-Type': 'application/x-www-form-urlencoded'}

In [381]:
# The date is still today, becaue we have only requested the page as if clicked the calendar button, 
# but not selected a previous date yet
pd.read_html(io.StringIO(str(page_request_calendar.text)) , thousands=None,
                          decimal=','  )[1][0][0]

'viernes, 16 de agosto de 2024'

In [382]:
# getting calendar page aspx params
calendar_EVENTVALIDATION , calendar_VIEWSTATE   = get_eventvalidation_viewstate (page_request_calendar)
calendar_VIEWSTATE[-10:] , calendar_EVENTVALIDATION[-10:]

('fg6jmLAcoH', '4PLI1nbkc=')

In [383]:
# calendar inspected.  the month is located at a td inside   <table id="Calendar1" 
cal_page_soup = bs(page_request_calendar.text)
cal_page_soup.find('table', attrs={'id': 'Calendar1'}).find_all('td')[2].text.split()

['agosto', 'de', '2024']

In [384]:
# Making sure calendar is actually at the right month
# getting the soup
if page_request_calendar.ok:
  cal_page_soup = bs(page_request_calendar.text)
else:
  print('Not Found. Status code was: ', page_request_calendar.status_code)

# extracting values cal month and year
current_cal_list = cal_page_soup.find('table', attrs={'id': 'Calendar1'}).find_all('td')[2].text.split()
current_cal_month = current_cal_list[0][:3]  #need only first 3 letters (to later conver to english)
current_cal_year = current_cal_list[2]

#validation
current_cal_month , current_cal_year

('ago', '2024')

In [385]:
# Converting month from spanish to english
# Dictionary of spanish months translated to english, commenting the ones that have no change
months_spa_to_eng = {'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun',\
                     'jul': 'Jul', 'ago': 'Aug','set': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec' }
for spa, eng in months_spa_to_eng.items():
    current_cal_month = current_cal_month.replace(spa, eng)

#Validation
current_cal_month

'Aug'

#### Function

In [585]:
def calendar_page_scrape ( days_backwards_to_scrape , base_EVENTVALIDATION , base_VIEWSTATE , payload = base_payload_json.copy() ):
    
  payload['__EVENTVALIDATION'] = base_EVENTVALIDATION
  payload['__VIEWSTATE'] = base_VIEWSTATE
  payload['__EVENTARGUMENT'] = days_backwards_to_scrape
  payload['__EVENTTARGET'] = ''  #thats how browser does it 
  # pretty = json.dumps(payload, indent = 1 )
  # print (pretty)
  # Performing the actual request.  Page with calendar is obtained with a POST request
  hdrs_post={'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        'Host' : f'{host}', 'Accept-Language': 'en-US,en;q=0.9,es;q=0.8,es-CR;q=0.7,de;q=0.6',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        # 'Content-Type': 'application/x-www-form-urlencoded'  #removed, requests library takes care of it
        }

  page_request_calendar = requests.post(url_page, headers=hdrs_post , data=payload )

  # getting calendar page aspx params
  calendar_EVENTVALIDATION , calendar_VIEWSTATE = get_eventvalidation_viewstate (page_request_calendar)
  
  # Making sure calendar is actually at the right month
  # getting the soup
  if page_request_calendar.ok:
    cal_page_soup = bs(page_request_calendar.text)
    # extracting values cal month and year
    current_cal_list = cal_page_soup.find('table', attrs={'id': 'Calendar1'}).find_all('td')[2].text.split()
    current_cal_month = current_cal_list[0][:3]  #need only first 3 letters (to later conver to english)
    current_cal_year = current_cal_list[2]
    
    # Converting month from spanish to english
    months_spa_to_eng = {'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun',\
                        'jul': 'Jul', 'ago': 'Aug','set': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec' }
    for spa, eng in months_spa_to_eng.items():
        current_cal_month = current_cal_month.replace(spa, eng)
    
    return calendar_EVENTVALIDATION , calendar_VIEWSTATE , current_cal_month, current_cal_year
  else:
    None

In [493]:
# function validation (calendar function does not need a value for eventargument, it can be empty)
calEV , calVS , calmon , calyear  = calendar_page_scrape ( 8950 , base_EVENTVALIDATION , base_VIEWSTATE , base_payload_json.copy() )
calEV[-5:] , calVS[-5:] , calmon, calyear

{
 "__EVENTTARGET": "",
 "__EVENTARGUMENT": 8950,
 "__VIEWSTATE": "/wEPDwUJMjQ0MTEwNDc3D2QWAgIDD2QWHgIBDw8WAh4EVGV4dAVKVGlwb3MgZGUgY2FtYmlvIGFudW5jaWFkb3MgZW4gdmVudGFuaWxsYSBwb3IgbG9zIGludGVybWVkaWFyaW9zIGNhbWJpYXJpb3NkZAIDDw8WAh8AZWRkAgUPDxYCHwAFR0VuIGNvbG9uZXMgY29zdGFycmljZW5zZXMgcG9yIGTDs2xhciBkZSBsb3MgRXN0YWRvcyBVbmlkb3MgZGUgQW3DqXJpY2EuZGQCBw8PFgIfAAUddmllcm5lcywgMTYgZGUgYWdvc3RvIGRlIDIwMjRkZAIJDzwrAAsCAA8WCB4IRGF0YUtleXMWAB4LXyFJdGVtQ291bnQCJh4VXyFEYXRhU291cmNlSXRlbUNvdW50AiYeCVBhZ2VDb3VudAIBZAEUKwAGPCsABAEAFgIeCkhlYWRlclRleHQFD1RpcG8gZGUgRW50aWRhZDwrAAQBABYCHwUFEkVudGlkYWQgQXV0b3JpemFkYTwrAAQBABYCHwUFBkNvbXByYTwrAAQBABYCHwUFBVZlbnRhPCsABAEAFgIfBQUVRGlmZXJlbmNpYWwgQ2FtYmlhcmlvPCsABAEAFgIfBQUWw5psdGltYSBBY3R1YWxpemFjacOzbhYCZg9kFkwCAQ9kFgxmDw8WAh8ABRBCYW5jb3MgcMO6YmxpY29zZGQCAQ8PFgIfAAVGQmFuY28gZGUgQ29zdGEgUmljYSAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGRkAgIPDxYCHwAFBjUxMiwwMGRkAgMPDxYCHwAFBjUyNiwwMGRkAgQPDxYCHwAFBTE0LDAwZGQCBQ8PFgIfAAUcMTUvMDgvMjAyNMKgw

('nbkc=', 'LAcoH', 'Aug', '2024')

#### Function 2

This one is very similar to the above, it is needed when calendar needs to be changed to previous month, because the aspx params/values are a bit different

In [584]:
def calendar_page_scrape_month_change ( day_evaluated , cal_EVENTVALIDATION , cal_VIEWSTATE , payload3 = base_payload_json.copy() ):
  date_converted_from_EVENTARGUMENT_test = date(2000, 1, 1) + timedelta (days = day_evaluated )
  _1st_of_month = date_converted_from_EVENTARGUMENT_test.replace(day=1)
  _1st_of_month_days = (_1st_of_month - date(2000, 1, 1)).days #convert back to #### format
  payload3['__EVENTVALIDATION'] = cal_EVENTVALIDATION
  payload3['__VIEWSTATE'] = cal_VIEWSTATE
  payload3['__EVENTARGUMENT'] = f'V{_1st_of_month_days}'  #this adds the V in V####
  del payload3['imgCalendario.x'] 
  del payload3['imgCalendario.y']  
  # pretty = json.dumps(payload3, indent = 1 )
  # print (pretty)
  # Performing the actual request.  Page with calendar is obtained with a POST request
  hdrs_post={'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        'Host' : f'{host}', 'Accept-Language': 'en-US,en;q=0.9,es;q=0.8,es-CR;q=0.7,de;q=0.6',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        # 'Content-Type': 'application/x-www-form-urlencoded'  #removed, requests library takes care of it
        }

  page_request_calendar = requests.post(url_page, headers=hdrs_post , data=payload3 )

  # getting calendar page aspx params
  calendar_EVENTVALIDATION , calendar_VIEWSTATE = get_eventvalidation_viewstate (page_request_calendar)
  
  # Making sure calendar is actually at the right month
  # getting the soup
  if page_request_calendar.ok:
    cal_page_soup = bs(page_request_calendar.text)
    # extracting values cal month and year
    current_cal_list = cal_page_soup.find('table', attrs={'id': 'Calendar1'}).find_all('td')[2].text.split()
    current_cal_month = current_cal_list[0][:3]  #need only first 3 letters (to later conver to english)
    current_cal_year = current_cal_list[2]
    
    # Converting month from spanish to english
    months_spa_to_eng = {'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun',\
                        'jul': 'Jul', 'ago': 'Aug','set': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec' }
    for spa, eng in months_spa_to_eng.items():
        current_cal_month = current_cal_month.replace(spa, eng)    
      
    return calendar_EVENTVALIDATION , calendar_VIEWSTATE , current_cal_month, current_cal_year
  else:
    # print('Not Found. Status code was: ', page_request_calendar.status_code)
    None
  

In [559]:
days_function_test = (date(2024,7,3) - date(2000, 1, 1)).days

In [560]:
# function2 validation . 
# eventargument must be V###, #### being the 1st of the month for that month. eg: for 2024jul03 (8950) it must be 8948
# EV and VS from calendar scrape are needed
calEV_new , calVS_new , calmon_new , calyear_new  = calendar_page_scrape_month_change ( days_function_test  , calEV , calVS, base_payload_json.copy() )
calEV_new[-5:] , calVS_new[-5:] , calmon_new, calyear_new

{
 "__EVENTTARGET": "Calendar1",
 "__EVENTARGUMENT": "V8948",
 "__VIEWSTATE": "/wEPDwUJMjQ0MTEwNDc3D2QWAgIDD2QWHgIBDw8WAh4EVGV4dAVKVGlwb3MgZGUgY2FtYmlvIGFudW5jaWFkb3MgZW4gdmVudGFuaWxsYSBwb3IgbG9zIGludGVybWVkaWFyaW9zIGNhbWJpYXJpb3NkZAIDDw8WAh8AZWRkAgUPDxYCHwAFR0VuIGNvbG9uZXMgY29zdGFycmljZW5zZXMgcG9yIGTDs2xhciBkZSBsb3MgRXN0YWRvcyBVbmlkb3MgZGUgQW3DqXJpY2EuZGQCBw8PFgIfAAUddmllcm5lcywgMTYgZGUgYWdvc3RvIGRlIDIwMjRkZAIJDzwrAAsCAA8WCB4IRGF0YUtleXMWAB4LXyFJdGVtQ291bnQCJh4VXyFEYXRhU291cmNlSXRlbUNvdW50AiYeCVBhZ2VDb3VudAIBZAEUKwAGPCsABAEAFgIeCkhlYWRlclRleHQFD1RpcG8gZGUgRW50aWRhZDwrAAQBABYCHwUFEkVudGlkYWQgQXV0b3JpemFkYTwrAAQBABYCHwUFBkNvbXByYTwrAAQBABYCHwUFBVZlbnRhPCsABAEAFgIfBQUVRGlmZXJlbmNpYWwgQ2FtYmlhcmlvPCsABAEAFgIfBQUWw5psdGltYSBBY3R1YWxpemFjacOzbhYCZg9kFkwCAQ9kFgxmDw8WAh8ABRBCYW5jb3MgcMO6YmxpY29zZGQCAQ8PFgIfAAVGQmFuY28gZGUgQ29zdGEgUmljYSAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGRkAgIPDxYCHwAFBjUxMiwwMGRkAgMPDxYCHwAFBjUyNiwwMGRkAgQPDxYCHwAFBTE0LDAwZGQCBQ8PFgIfAAUcMTUvM

('wkKA=', 'Www==', 'Jul', '2024')

### Changing calendar page to prev month if necessary

In [435]:
base_EVENTVALIDATION[-5:] , base_VIEWSTATE[-5:] , calEV[-5:] , calVS[-5:], calmon, calyear

('aco0x', 'cRg8=', 'nbkc=', 'LAcoH', 'Aug', '2024')

In [446]:
base_payload_json

{'__EVENTTARGET': 'Calendar1',
 '__EVENTARGUMENT': '',
 '__VIEWSTATE': '',
 '__VIEWSTATEGENERATOR': '5CF5411C',
 '__EVENTVALIDATION': '',
 'imgCalendario.x': '7',
 'imgCalendario.y': '8',
 'CtrlBuscar2:txtPalabras': ' '}

In [562]:
# Purpose of this cell is to quickly calculare the #### date/day number for testing
# In the next section we would work with days in relation to 2000-1-1, so the validation must be done 
# using that kind of number

#same as  days_function_test  defined above
days_function_test 

8950

In [420]:
# # This was need as pc where this was developed was using es_US, and spanish was intefering with
# # dates generation names.  added to top of code
# locale.setlocale(locale.LC_TIME, ('en_US', 'UTF-8'))

In [563]:
# We must learn to convert that number to date object in order to get its month and year so we can compare
# against the one we got

# The way to do that is to use a days timedelta summed on top of 2000-1-1
date_converted_from_EVENTARGUMENT = date(2000, 1, 1) + timedelta (days = days_function_test )

date_converted_from_EVENTARGUMENT, date_converted_from_EVENTARGUMENT.strftime('%b'), str(date_converted_from_EVENTARGUMENT.year)

(datetime.date(2024, 7, 3), 'Jul', '2024')

In [416]:
# # But date_converted_from_EVENTARGUMENT.strftime('%b')  is the mmm month in spanish.  need to convert that \
# # to english to make a proper comparison agains calmon which is in englishc

# EVENTARGUMENT_month_test = date_converted_from_EVENTARGUMENT.strftime('%b')

# # Converting month from spanish to english
# months_spa_to_eng = {'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun',\
#                      'jul': 'Jul', 'ago': 'Aug','set': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec' }
# for spa, eng in months_spa_to_eng.items():
#     current_cal_month = EVENTARGUMENT_month_test.replace(spa, eng)
    
# EVENTARGUMENT_month_test

In [564]:
# this piece of code should help with the comparison then, when its time to iterate for many days

# EVENTARGUMENT_days = 8978
EVENTARGUMENT_days = days_function_test 
date_converted_from_EVENTARGUMENT = date(2000, 1, 1) + timedelta (days = EVENTARGUMENT_days)
EVENTARGUMENT_month = date_converted_from_EVENTARGUMENT.strftime('%b')
EVENTARGUMENT_year = str(date_converted_from_EVENTARGUMENT.year)  
print(calmon , calyear , EVENTARGUMENT_month , EVENTARGUMENT_year)
if [ calmon , calyear] == [EVENTARGUMENT_month , EVENTARGUMENT_year]:
  print('No need to change calendar month/year')
else:
  print('Calendar month/year must be adjusted')
# calendar args must be sent again to function calendar_page_scrape with latest calEV and calVS\
# and eventargument as 'V####' of the first day of the same month as EVENTARGUMENT_year, and obtain new 
# calEV and calVS

Aug 2024 Jul 2024
Calendar month/year must be adjusted


In [566]:
# This is the calendar change going back to previous month, overwriting the calEV and calVS to finally
# get the data from the desired date in the past
calEV , calVS , calmon , calyear  = calendar_page_scrape_month_change ( days_function_test , calEV , calVS, base_payload_json.copy() )
calEV[-5:] , calVS[-5:] , calmon , calyear 

{
 "__EVENTTARGET": "Calendar1",
 "__EVENTARGUMENT": "V8948",
 "__VIEWSTATE": "/wEPDwUJMjQ0MTEwNDc3D2QWAgIDD2QWHgIBDw8WAh4EVGV4dAVKVGlwb3MgZGUgY2FtYmlvIGFudW5jaWFkb3MgZW4gdmVudGFuaWxsYSBwb3IgbG9zIGludGVybWVkaWFyaW9zIGNhbWJpYXJpb3NkZAIDDw8WAh8AZWRkAgUPDxYCHwAFR0VuIGNvbG9uZXMgY29zdGFycmljZW5zZXMgcG9yIGTDs2xhciBkZSBsb3MgRXN0YWRvcyBVbmlkb3MgZGUgQW3DqXJpY2EuZGQCBw8PFgIfAAUddmllcm5lcywgMTYgZGUgYWdvc3RvIGRlIDIwMjRkZAIJDzwrAAsCAA8WCB4IRGF0YUtleXMWAB4LXyFJdGVtQ291bnQCJh4VXyFEYXRhU291cmNlSXRlbUNvdW50AiYeCVBhZ2VDb3VudAIBZAEUKwAGPCsABAEAFgIeCkhlYWRlclRleHQFD1RpcG8gZGUgRW50aWRhZDwrAAQBABYCHwUFEkVudGlkYWQgQXV0b3JpemFkYTwrAAQBABYCHwUFBkNvbXByYTwrAAQBABYCHwUFBVZlbnRhPCsABAEAFgIfBQUVRGlmZXJlbmNpYWwgQ2FtYmlhcmlvPCsABAEAFgIfBQUWw5psdGltYSBBY3R1YWxpemFjacOzbhYCZg9kFkwCAQ9kFgxmDw8WAh8ABRBCYW5jb3MgcMO6YmxpY29zZGQCAQ8PFgIfAAVGQmFuY28gZGUgQ29zdGEgUmljYSAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGRkAgIPDxYCHwAFBjUxMiwwMGRkAgMPDxYCHwAFBjUyNiwwMGRkAgQPDxYCHwAFBTE0LDAwZGQCBQ8PFgIfAAUcMTUvM

('wkKA=', 'Www==', 'Jul', '2024')

### Getting previous date page

The new POST request must be made with updated aspx EVENTVALIDATION and VIEWSTATE values

In [569]:
# using a new payload var 
payload_json2 = base_payload_json.copy()
# this one must not have the imgCalendario keys.  Deleting them
del payload_json2['imgCalendario.x'] 
del payload_json2['imgCalendario.y']
# argument that controls the date, as described at the project's intro
# past_date_to_scrape = date(2024, 8, 1) #result is like  datetime.date(2024, 8, 1)
# past_date_to_scrape_inDays = (past_date_to_scrape - date(2000, 1, 1)).days #result like 8###
past_date_to_scrape_inDays = days_function_test
# __EVENTVALIDATION and __VIEWSTATE populated with info from previous section, and __EVENTARGUMENT from
#info of past date to be scraped
# payload_json2['__EVENTVALIDATION'] = calendar_EVENTVALIDATION
payload_json2['__EVENTVALIDATION'] = calEV
# payload_json2['__VIEWSTATE'] = calendar_VIEWSTATE
payload_json2['__VIEWSTATE'] = calVS
payload_json2['__EVENTARGUMENT'] = past_date_to_scrape_inDays

#
for k in  ['__EVENTVALIDATION','__VIEWSTATE'] :
  val = payload_json2[k][-5:]
  print(f'{k} : {val}')

__EVENTVALIDATION : wkKA=
__VIEWSTATE : Www==


In [570]:
# Performing the actual request.  Previous day's data is done with a POST request
# using same headers already stored before, and new payload jason var

page_request_prevdate = requests.post(url_page, headers=hdrs_post , data=payload_json2 )

page_request_prevdate.status_code, page_request_prevdate.reason

(200, 'OK')

In [571]:
# 
pd.read_html(io.StringIO(str(page_request_prevdate.text)) , thousands=None, decimal=','  )[1][0][0]

'miércoles, 3 de julio de 2024'

#### Function

In [399]:
base_payload_json

{'__EVENTTARGET': 'Calendar1',
 '__EVENTARGUMENT': '',
 '__VIEWSTATE': '',
 '__VIEWSTATEGENERATOR': '5CF5411C',
 '__EVENTVALIDATION': '',
 'imgCalendario.x': '7',
 'imgCalendario.y': '8',
 'CtrlBuscar2:txtPalabras': ' '}

In [572]:
def prevday_page_scrape (specificDay, cal_EVENTVALIDATION, cal_VIEWSTATE, payload2 = base_payload_json.copy() ):
  # using a new payload var 
    # this one must not have the imgCalendario keys.  Deleting them
  del payload2['imgCalendario.x'] 
  del payload2['imgCalendario.y']
  # __EVENTVALIDATION and __VIEWSTATE populated with info from cal page
  payload2['__EVENTVALIDATION'] = cal_EVENTVALIDATION
  payload2['__VIEWSTATE'] = cal_VIEWSTATE
  payload2['__EVENTARGUMENT'] = specificDay
  # print(payload2)
    # Performing the actual request.  Page with calendar is obtained with a POST request
  hdrs_post={'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        'Host' : f'{host}', 'Accept-Language': 'en-US,en;q=0.9,es;q=0.8,es-CR;q=0.7,de;q=0.6',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        # 'Content-Type': 'application/x-www-form-urlencoded'  #removed, requests library takes care of it
        }

  page_request_prevdate = requests.post(url_page, headers=hdrs_post , data=payload2 )

  return page_request_prevdate 



In [401]:
(date(2024,8,1)- date(2000, 1, 1)).days

8979

In [573]:
# Function validation
page_request_prevdate2 = prevday_page_scrape( days_function_test , calEV , calVS, base_payload_json.copy() )

In [574]:
pd.read_html(io.StringIO(str(page_request_prevdate2.text)) , thousands=None, decimal=',' )[1][0][0]

'miércoles, 3 de julio de 2024'

In [575]:
page_request_prevdate2.text[40330:41000]

' align="left">&nbsp;</td><td align="left">Banco Nacional de Costa Rica                                          </td><td align="right">517,50</td><td align="right">531,00</td><td align="right">13,50</td><td>03/07/2024\xa0\xa0\xa0\xa008:08 a.m.</td>\r\n\t</tr><tr align="right" style="color:Black;background-color:White;font-family:Arial;font-size:10pt;">\r\n\t\t<td align="left" style="color:White;">&nbsp;</td><td align="left">Banco Popular y de Desarrollo Comunal                                 </td><td align="right">518,00</td><td align="right">532,00</td><td align="right">14,00</td><td>27/06/2024\xa0\xa0\xa0\xa012:28 p.m.</td>\r\n\t</tr><tr align="right" style="color:Black;background-color:#DFED'

### Building the dataframe from scraped html

In [576]:
past_date_to_scrape , past_date_to_scrape.strftime('%Y-%m-%d') , (past_date_to_scrape - date(2000, 1, 1)).days

(datetime.date(2024, 8, 1), '2024-08-01', 8979)

In [577]:
## the function from todays exc works...except for the date column which still displays today's date
dol_exch_ent_df_build(page_request_prevdate).head(3)
# so..need new function based on the previous one for generating the df

,date,ent_type,ent_name,dollar_buy,dollar_sale,b_s_diff,previous_updt
0,2024-08-16,Bancos públicos,Banco de Costa Rica,519.0,533.0,14.0,2024-07-03 09:01:00
1,2024-08-16,Bancos públicos,Banco Nacional de Costa Rica,517.5,531.0,13.5,2024-07-03 08:08:00
2,2024-08-16,Bancos públicos,Banco Popular y de Desarrollo Comunal,518.0,532.0,14.0,2024-06-27 12:28:00


#### Function

In [578]:
## building a new function for prev date df build, based on todays df function
def dol_exch_ent_df_build_prevday (pagereq , dateYMD):
  df_tempprev = dol_exch_ent_df_build(pagereq)  
  df_tempprev['date']=dateYMD  # updates date to the one of the day being evaluated
  df_tempprev['date']=pd.to_datetime(df_tempprev['date'])
  return df_tempprev

In [579]:
# function validation

# dol_exch_ent_df_build_prevday( page_request_prevdate, '2024-08-01' ).head(3)
dol_exch_ent_df_build_prevday( page_request_prevdate, past_date_to_scrape.strftime('%Y-%m-%d') ).head(3)

,date,ent_type,ent_name,dollar_buy,dollar_sale,b_s_diff,previous_updt
0,2024-08-01,Bancos públicos,Banco de Costa Rica,519.0,533.0,14.0,2024-07-03 09:01:00
1,2024-08-01,Bancos públicos,Banco Nacional de Costa Rica,517.5,531.0,13.5,2024-07-03 08:08:00
2,2024-08-01,Bancos públicos,Banco Popular y de Desarrollo Comunal,518.0,532.0,14.0,2024-06-27 12:28:00


## 4.Full iterative scraping entities dollar exchange rate from many different days before today

Now that it is known how the past day scraping works, its time to scrape many different days, in bulk.

Steps:

4.1. Read Calendar page and extract aspx params, and also prev days's month \
4.2. If prev day's month matches the calendar month, continue to 4.3.  If not, request cal page for prev month\
---4.2.1 Update aspx params\
4.3. Read desired date page and extract html using requests\
4.4. Build dataframe and clean it, and add it to a definitive df for all the dates iterated


In [86]:
base_EVENTVALIDATION[-5:], base_VIEWSTATE[-5:] , calEV[-5:] , calVS[-5:]

('1PAiM', '3eg==', 'jD3w=', 'Wlbs=')

In [611]:
# Performing the actual request, recursively.  Previous day's data is done with a POST request

# Defining the number of previous days to scrape is defined, and today's eventarg days number
PREV_DAYS_TO_SCRAPE = 20
todays_eventArgument = (datetime.today().date() - date(2000, 1, 1)).days # number like 8###
oldest_date_to_extract = (datetime.today().date() - timedelta(days=PREV_DAYS_TO_SCRAPE)).strftime('%Y-%m-%d')

# initializing dataframe to store all the data
df_dol_ent_prev_days = pd.DataFrame()

for d in range(1,(PREV_DAYS_TO_SCRAPE+1)):
  past_eventArgument = todays_eventArgument - d  # number like 8###
  # date_evaluated in YYYY-mm-dd format
  date_evaluated = (datetime.today().date() - timedelta(days=d)).strftime('%Y-%m-%d')
  print(f'{date_evaluated}:Payload EVNTARG {past_eventArgument}' ,  end = ' -- '  )
# 4.1 Getting calendar page and its needed values
  calEV , calVS , calmon , calyear  = calendar_page_scrape ( '' , base_EVENTVALIDATION , base_VIEWSTATE, base_payload_json.copy() )
  print(f'{calEV[-5:]} , {calVS[-5:]} , {calmon}, {calyear}' ,  end = ' -- '  )

# 4.2 Check if calendar for current evaluated date maches that date's month
  date_converted_from_EVENTARGUMENT = date(2000, 1, 1) + timedelta (days = past_eventArgument )
  EVENTARGUMENT_month = date_converted_from_EVENTARGUMENT.strftime('%b')
  EVENTARGUMENT_year = str(date_converted_from_EVENTARGUMENT.year)
  print( calmon , calyear, EVENTARGUMENT_month , EVENTARGUMENT_year)
  if [ calmon , calyear ] != [EVENTARGUMENT_month , EVENTARGUMENT_year]:
    # 4.2.1 need to update calendar to prev month
    print('Calendar mon/Y must be adjusted')
    # function2 of calendar comes into play to update calEV and calVS from cal of previous month
    calEV , calVS , calmon_new , calyear_new  = calendar_page_scrape_month_change (past_eventArgument, calEV, calVS, base_payload_json.copy() )
    calEV[-5:] , calVS[-5:] , calmon_new, calyear_new
  else:
    print('No need to change calendar mon/Y')    
# 4.3 Scraping the page for date at present iteration    
  page_request_prev_date = prevday_page_scrape( past_eventArgument, calEV, calVS, base_payload_json.copy() )
  print(page_request_prev_date.status_code)
  pd.read_html(io.StringIO(str(page_request_prevdate2.text)) , thousands=None, decimal=',' )[1][0][0]
  
  # 4.4. Generate DF.  one of its inputs is  date_evaluated  above, to populate date column
  df_dol_ent_temp = dol_exch_ent_df_build_prevday( page_request_prev_date, date_evaluated )
  display(df_dol_ent_temp.head(1))
  # add evaluated date df to the definitive df
  df_dol_ent_prev_days = pd.concat ([ df_dol_ent_temp , df_dol_ent_prev_days])
  del(df_dol_ent_temp)


# Final report of the recursive extraction
earliest_date_stored = df_dol_ent_prev_days['date'].unique().min()
# of days succesfully extracted
succ_days = len(df_dol_ent_prev_days['date'].unique())
print(f'\n>>>Done, dataframe stored from yesterday and back to {earliest_date_stored} \
with {df_dol_ent_prev_days.shape[0]} rows.\n{succ_days} days were successfully extracted')


2024-08-15:Payload EVNTARG 8993 -- nbkc= , LAcoH , Aug, 2024 -- Aug 2024 Aug 2024
No need to change calendar mon/Y
200


,date,ent_type,ent_name,dollar_buy,dollar_sale,b_s_diff,previous_updt
0,2024-08-15,Bancos públicos,Banco de Costa Rica,512.0,526.0,14.0,2024-08-15 00:03:00


2024-08-14:Payload EVNTARG 8992 -- nbkc= , LAcoH , Aug, 2024 -- Aug 2024 Aug 2024
No need to change calendar mon/Y
200


,date,ent_type,ent_name,dollar_buy,dollar_sale,b_s_diff,previous_updt
0,2024-08-14,Bancos públicos,Banco de Costa Rica,511.0,525.0,14.0,2024-08-14 12:58:00


2024-08-13:Payload EVNTARG 8991 -- nbkc= , LAcoH , Aug, 2024 -- Aug 2024 Aug 2024
No need to change calendar mon/Y
200


...

2024-07-28:Payload EVNTARG 8975 -- nbkc= , LAcoH , Aug, 2024 -- Aug 2024 Jul 2024
Calendar mon/Y must be adjusted
200


,date,ent_type,ent_name,dollar_buy,dollar_sale,b_s_diff,previous_updt
0,2024-07-28,Bancos públicos,Banco de Costa Rica,514.0,528.0,14.0,2024-07-27 00:02:00


2024-07-27:Payload EVNTARG 8974 -- nbkc= , LAcoH , Aug, 2024 -- Aug 2024 Jul 2024
Calendar mon/Y must be adjusted
200


,date,ent_type,ent_name,dollar_buy,dollar_sale,b_s_diff,previous_updt
0,2024-07-27,Bancos públicos,Banco de Costa Rica,514.0,528.0,14.0,2024-07-27 00:02:00



>>>Done, dataframe stored from yesterday and back to 2024-07-27 00:00:00 with 760 rows.
20 days were successfully extracted


In [612]:
# validating dtype of vars.   Also, #of rows must be 38 * number of days scraped
df_dol_ent_prev_days.info()

<class 'pandas.core.frame.DataFrame'>
Index: 760 entries, 0 to 37
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           760 non-null    datetime64[ns]
 1   ent_type       760 non-null    object        
 2   ent_name       760 non-null    object        
 3   dollar_buy     760 non-null    float64       
 4   dollar_sale    760 non-null    float64       
 5   b_s_diff       760 non-null    float64       
 6   previous_updt  760 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(2)
memory usage: 47.5+ KB


In [616]:
display(df_dol_ent_prev_days.head(2) ), display(df_dol_ent_prev_days.tail(2))

,date,ent_type,ent_name,dollar_buy,dollar_sale,b_s_diff,previous_updt
0,2024-07-27,Bancos públicos,Banco de Costa Rica,514.0,528.0,14.0,2024-07-27 00:02:00
1,2024-07-27,Bancos públicos,Banco Nacional de Costa Rica,514.0,528.0,14.0,2024-07-26 15:58:00


,date,ent_type,ent_name,dollar_buy,dollar_sale,b_s_diff,previous_updt
36,2024-08-15,Puestos de Bolsa,PB Inversiones SAMA,507.0,523.0,16.0,2024-08-14 13:50:00
37,2024-08-15,Puestos de Bolsa,"Popular Valores, Puesto de Bolsa",513.0,527.0,14.0,2024-08-14 10:54:00


(None, None)

In [617]:
# validating earliest and latest days
df_dol_ent_prev_days['date'].unique().min() , df_dol_ent_prev_days['date'].unique().max()

(Timestamp('2024-07-27 00:00:00'), Timestamp('2024-08-15 00:00:00'))

In [ ]:
# df_dol_ent_prev_days

## Adding todays + prev days values to master updated DF

In [618]:
df_dol_ent = pd.concat  ( [ df_dol_ent_prev_days , df_dol_ent_today ] )
del(df_dol_ent_prev_days)
del(df_dol_ent_today)

In [619]:
# showing limit between last day from prev days extraction, and todays extraction
df_dol_ent.tail(40).head(4)

,date,ent_type,ent_name,dollar_buy,dollar_sale,b_s_diff,previous_updt
36,2024-08-15,Puestos de Bolsa,PB Inversiones SAMA,507.0,523.0,16.0,2024-08-14 13:50:00
37,2024-08-15,Puestos de Bolsa,"Popular Valores, Puesto de Bolsa",513.0,527.0,14.0,2024-08-14 10:54:00
0,2024-08-16,Bancos públicos,Banco de Costa Rica,512.0,526.0,14.0,2024-08-15 00:03:00
1,2024-08-16,Bancos públicos,Banco Nacional de Costa Rica,508.0,522.0,14.0,2024-08-14 12:39:00


In [620]:
df_dol_ent['date'].unique().min() ,  df_dol_ent['date'].unique().max()

(Timestamp('2024-07-27 00:00:00'), Timestamp('2024-08-16 00:00:00'))

In [621]:
df_dol_ent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 798 entries, 0 to 37
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           798 non-null    datetime64[ns]
 1   ent_type       798 non-null    object        
 2   ent_name       798 non-null    object        
 3   dollar_buy     798 non-null    float64       
 4   dollar_sale    798 non-null    float64       
 5   b_s_diff       798 non-null    float64       
 6   previous_updt  798 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(2)
memory usage: 49.9+ KB


In [233]:
# df_dol_ent

## Backing up entities dollar exchange rate values at dataset parquet file

In [622]:
# Now lets update the exchange data file, using the updated dataframe

df_dol_ent.to_parquet(f'{DATASETS_PATH}/{CURRENT_DATASET_BASEFILENAME}.parquet')
current_dataset_file_creationTstamp = datetime.fromtimestamp (os.path.getctime(f'{DATASETS_PATH}/{CURRENT_DATASET_BASEFILENAME}.parquet')).strftime('%Y-%m-%d %H:%M')
current_dataset_file_size = os.path.getsize(f'{DATASETS_PATH}/{CURRENT_DATASET_BASEFILENAME}.parquet')
print(f'Current dataset file has been updated: {DATASETS_PATH}/{CURRENT_DATASET_BASEFILENAME}.parquet \
      \n--creation time: {current_dataset_file_creationTstamp} -- filesize: {current_dataset_file_size} bytes \
      \nCurrent date-time is:', datetime.today().strftime('%Y-%m-%d %H:%M'))

Current dataset file has been updated: ./datasets/bccr_dol_exch_entities.parquet       
--creation time: 2024-08-16 04:24 -- filesize: 11966 bytes       
Current date-time is: 2024-08-16 04:24
